In [2]:
!pip install twitter==2.0a2
import pandas as pd
import twitter
from time import time, sleep
import pandas as pd
import json

In [3]:
from twitter import OAuth2

def bearers_to_apis(bearers,version) :
    """
    Entrée :
        bearers : une liste de bearers dont la structure est identique à celle de bearers.json
        version : la version de l'api twitter voulue (au format string)
    Sortie :
        apis_list : une liste d'api du module twitter
    """
    apis_list = []
    for bear in bearers : 
        b_auth = OAuth2(bearer_token= bear['bearer'])
        b_to_api = twitter.Twitter(auth=b_auth,api_version=version)
        apis_list.append(b_to_api)
    return(apis_list)

In [4]:
def following_scrapper(ids_list,following_number,apis_list,save) :
    """
    Entrée :
        ids_list : la liste des ids des utilisateurs twitter dont il faut récupérer la liste des comptes following (ids au format string)
        following_number : la liste du nombre de following d'un utilisateur selon sont indice dans ids_list
        apis_list : la liste des apis de la librairie twitter, de version 1.1 dont on va se servir pour récupérer la liste des following
        save : nom du fichier de sauvegarde, au format str
    Sortie : 
        dict_following : un dictionnaire où chaque clé est l'id d'un utilisateur twitter et qui renvoie la liste de ses following 
    """
    n = len(apis_list) #nombre total d'apis disponibles
    i = 0 #id de l'api utilisée
    dict_following = dict()
    for k in range(len(ids_list)) :
        user = ids_list[k] #id de l'utilisateur dont on récupère les followings
        if following_number[k] == 0 : #il n'y a tout simplement pas de following à récupérer
            dict_following[user] = []
            continue
        following = [] #liste des following de l'utilisateur actuel
        page = '-1' #page de la liste des following que l'on regarde, initialement '-1'
        while page != '0' : #la page "vide" dont le numéro est 0 est celle qui conclue la liste des following
            try :
                f_request = apis_list[i].friends.ids(user_id = user,stringify_ids = True,cursor = page,count = 5000)
                page = f_request['next_cursor_str'] #Changement de la page
                following += f_request['ids'] #Ajout des followings ajoutés
                if f_request.rate_limit_remaining == 0 : #Il n'y a plus de requêtes disponibles pour l'API
                    print('Plus de requêtes disponibles, changement de l\'API utilisée ['+str(i+1)+'/'+str(n)+']')
                    i = (i+1)%n
                    if i == 0 : #si au moment de changer d'api, on revient à l'api initiale, on attend le temps qu'il faut pour réinitialiser le nombre de requêtes disponibles
                        print('La première API n\'a pas eu le temps d\'être réinitialisée, veuillez attendre 15 minutes')
                        sleep(15*60) #les fenêtres d'utilisation des requêtes sont de 15 minutes
            except Exception as e : #Dans le cas d'une erreur
                if e.e.code == 401 : #Cette erreur arrive souvent dans le cas où l'utilisateur recherché a été suspendu
                    print('Le compte de cet utilisateur a été suspendu')
                    break
                if e.e.code == 404 : #Cette erreur arrive souvent dans le cas où l'utilisateur n'existe plus
                    print('Le compte a été supprimé')
                    break
                #Dans le doute on change d'api
                print('Il y a eu une erreur dans l\'utilisation de l\'API, changement de l\'API ['+str(i+1)+'/'+str(n)+']' + ' ('+ str(e)+')')
                i = (i+1)%n
                if i == 0 : #si au moment de changer d'api, on revient à l'api initiale, on attend le temps qu'il faut pour réinitialiser le nombre de requêtes disponibles 
                    print('La première API n\'a pas eu le temps d\'être réinitialisée, veuillez attendre 15 minutes')
                    sleep(15*60) #les fenêtres d'utilisation des requêtes sont de 15 minutes
                continue
        dict_following[user] = following #Mise à jour du dictionnaire
        print('Avancement : '+str(k+1)+'/'+str(len(ids_list))+' terminé')
        if k%100 == 0 : #On sauvegarde tout les 100 utilisateurs
            with open(save + '.json', 'w') as fp:
                json.dump(dict_following, fp)
    with open(save+'.json', 'w') as fp: #enregistrement final
                json.dump(dict_following, fp)
    return(dict_following)


In [5]:
f = open('bearers.json')
data = json.load(f)
f.close()
apis = bearers_to_apis(bearers = data,version = "1.1")

f = open('users.json')
users = json.load(f)
f.close()

In [6]:
ids_users = [users['id'][str(k)] for k in list(users['id'].keys())]
nb_following = [users['public_metrics'][str(k)]['following_count'] for k in list(users['id'].keys())]

In [ ]:
followings = following_scrapper(ids_list=ids_users,following_number=nb_following,apis_list=apis,save = 'lst_following')